In [6]:
import numpy as np
import pandas as pd

### Load data

In [7]:
df_C_raw = pd.read_excel("hierarchy_org_position_id.xlsx", sheet_name="CDIO_COO_DIV PRIVATE BANKING")
df_P_raw = pd.read_excel("hierarchy_org_position_id.xlsx", sheet_name="Private Banking")
df_OP = pd.read_excel("hierarchy_org_position_id.xlsx", sheet_name="Organizational Position")

df_Employees = pd.read_excel("Data.xlsx", sheet_name="Employees")
df_Domain = pd.read_excel("Data.xlsx", sheet_name="Knowledge structure")
df_Assessment = pd.read_excel("Data.xlsx", sheet_name="Knowledge assessment", na_values = "NA")
df_Organization = pd.read_excel("Data.xlsx", sheet_name="Organizational structure", na_values="NA")
df_Persona = pd.read_excel("Data.xlsx", sheet_name="Personas")

## EDA

### Employee Data

In [ ]:
df_Employees.head()

,ID_EMPLOYEE,SEX,AGE,ID_FRAME,FLAG_RESPONSABILE,ID_ORGANIZATIONAL_POSITION
0,2,M,60,3A/4,#,10134
1,3,F,49,QD/1,#,10130
2,4,F,60,QD/4,#,10201
3,5,M,53,DR/0,#,1648
4,6,F,52,QD/4,X,55


#### Number of employees

In [ ]:
len(df_Employees.ID_EMPLOYEE.unique())

64998

#### Proportion of males to females

In [ ]:
df_Employees['SEX'].value_counts(normalize = True)

F    0.510238
M    0.489762
Name: SEX, dtype: float64

#### Distribution of age

In [ ]:
df_Employees['AGE'].value_counts(normalize = True)

54    0.045924
49    0.045785
57    0.044478
55    0.043032
52    0.041986
58    0.041940
53    0.041802
56    0.040525
59    0.040218
51    0.039726
50    0.039433
60    0.038926
48    0.036050
47    0.031621
46    0.030729
61    0.029129
45    0.028268
40    0.028099
41    0.027960
39    0.027483
44    0.026653
42    0.024900
43    0.024331
38    0.023592
62    0.020163
37    0.019501
36    0.017317
63    0.015780
35    0.013288
64    0.010996
34    0.009966
33    0.007736
32    0.007705
65    0.006967
31    0.005952
30    0.005075
29    0.004076
28    0.003091
66    0.002799
67    0.002245
27    0.002030
26    0.001046
25    0.000615
24    0.000400
68    0.000323
23    0.000261
22    0.000046
71    0.000015
21    0.000015
Name: AGE, dtype: float64

#### Potential data quality issue

ID_EMPLOYEE is not unique on ID_ORGANIZATIONAL_POSITION


In [ ]:
len(df_Employees.index) == len(df_Employees.ID_EMPLOYEE.unique())

counts = df_Employees['ID_EMPLOYEE'].value_counts()
org_issue = df_Employees[df_Employees['ID_EMPLOYEE'].isin(counts.index[counts > 1])]

id_not_unk = org_issue.ID_EMPLOYEE.unique()
org_pos = org_issue.ID_ORGANIZATIONAL_POSITION.unique()

Only a handful of organizational positions are affected

In [ ]:
print(org_pos)

[4593 5051 4596 5107 4607 6145]


These organizational positions are linked to one Persona, but different provinces and regions, which means that a handful of employees are linked to multiple regions and provinces

### Organizational Data

In [ ]:

df_Organization.head()

,ID_ORGANIZATIONAL_POSITION,PROVINE,REGION,ID_PERSONAS
0,1,Imperia,Liguria,P00029
1,2,Novara,Piemonte,P00029
2,3,Biella,Piemonte,P00029
3,4,Milano,Lombardia,P00029
4,5,Milano,Lombardia,P00029


#### Personas
There are 36 unique personas (One is NA). Personas are not represented in every region and therefore also not represented in every province.

In [ ]:
df_Organization.ID_PERSONAS.unique()

array(['P00029', 'P00030', 'P00007', 'P00035', 'P00032', 'P00023',
       'P00008', 'P00014', 'P00036', 'P00009', 'P00001', 'P00027',
       'P00012', 'P00018', 'P00026', nan, 'P00024', 'P00010', 'P00004',
       'P00028', 'P00011', 'P00034', 'P00019', 'P00003', 'P00021',
       'P00002', 'P00025', 'P00022', 'P00033', 'P00006', 'P00020',
       'P00015', 'P00031', 'P00005', 'P00017', 'P00013', 'P00016'],
      dtype=object)

In [ ]:
len(df_Organization.ID_PERSONAS.unique())

37

#### Regions and provinces
Provinces are nested inside of regions. There are 21 regions and 108 provinces represented in the data.
The following dataset

In [ ]:
region = df_Organization[["REGION", "PROVINE"]]
region = region.drop_duplicates()
region.sort_values(by=['REGION'], inplace = True)
print(region)

          REGION   PROVINE
4119     Abruzzo  L'Aquila
4073     Abruzzo    Chieti
4035     Abruzzo    Teramo
102      Abruzzo   Pescara
5934  Basilicata   Potenza
...          ...       ...
18        Veneto   Venezia
17        Veneto   Treviso
113       Veneto    Padova
187       Veneto   Belluno
96        Veneto    Rovigo

[108 rows x 2 columns]


#### Data quality issue from employees data
The following filtered table shows records that would be affected by the data quality issue in the employees data if the two tables were to be joined. The only persona affected by this issue is P00032 (Credit & Business Finance).

In [ ]:

df_Organization[df_Organization.ID_ORGANIZATIONAL_POSITION.isin(org_pos)]

,ID_ORGANIZATIONAL_POSITION,PROVINE,REGION,ID_PERSONAS
4592,4593,Pistoia,Toscana,P00032
4595,4596,Viterbo,Lazio,P00032
4606,4607,Catanzaro,Calabria,P00032
5050,5051,Perugia,Umbria,P00032
5106,5107,Pistoia,Toscana,P00032
6144,6145,Bari,Puglia,P00032


### Knowledge Domain Data
There are 130 knowledge domains and 43 knowledge areas. There are two knowledge domains without knowledge areas.

In [10]:
df_Domain.head()

,ID_KNOWLEDGE_DOMAIN,DESCR_KNOWLEDGE_DOMAIN,ID_KNOWLEDGE_AREA,DESCR_KNOWLEDGE_AREA
0,C22D02,Data Analysis,AREAC022,Data governance
1,C03D01,Operativit√† di sportello,AREAC003,Conoscenza di Prodotti e Servizi bancari
2,C11D02,Monitoraggio del credito,AREAC011,Analisi e Gestione del credito
3,C23D02,Matematica finanziaria,AREAC023,Modelli quantitativi
4,C36D01,Innovation management,AREAC036,Digital


In [16]:
len(df_Domain.ID_KNOWLEDGE_DOMAIN.unique())

130

In [17]:
len(df_Domain.ID_KNOWLEDGE_AREA.unique())

43

### Knowledge Assessment Data
A few employees (106) are not represented in the knowledge assessment data. 

In [19]:
df_Assessment.head()

,ID_EMPLOYEE,ID_MANAGER,ID_KNOWLEDGE_DOMAIN,SCORE_AUTODETECTION,SCORE_VALIDATION
0,63374,767,C05_1D01,1,NaN
1,39838,42166,C11D02,1,NaN
2,27198,8078,C15D03,1,1.0
3,28622,20863,C23D02,1,NaN
4,20494,25046,C28D04,1,NaN


In [15]:
print(len(df_Assessment.ID_EMPLOYEE.unique()))
print(len(df_Employees.ID_EMPLOYEE.unique()))

64892
64998


In [34]:
df = df_Assessment[df_Assessment['SCORE_VALIDATION'].notna()]

(df['SCORE_AUTODETECTION'] == df['SCORE_VALIDATION']).value_counts(normalize = True)

True     0.868949
False    0.131051
dtype: float64

There were 52310 instances where employees did not rank them selves the same as their manager in a particular skill. 54% ranked themselves higher and 46% ranked themselves lower.

In [27]:
not_eq = df[df['self_eq_valid'] == False]
not_eq['self_gt_valid'].value_counts(normalize = True)

True     0.538176
False    0.461824
Name: self_gt_valid, dtype: float64

In 73% of cases where the self-assessment did not match the managerial assessment the assessment value only differed by 1. This indicates that for the given sample, people were generally very accurate in their self-assessment.

In [33]:
(not_eq['SCORE_AUTODETECTION'] - not_eq['SCORE_VALIDATION']).value_counts(normalize = True, sort = True)

 1.0    0.382489
-1.0    0.355611
 2.0    0.101032
-2.0    0.082413
 3.0    0.039266
-3.0    0.020264
 4.0    0.015389
-4.0    0.003537
dtype: float64

## Create Network Data

The following block of code is transforming the variables listed in the uog_id array from wide to long, binding the two dataframes together and taking the unique value of the resulting 'value' column. The unique values from the value column are used to subset df_OP_filter in the following step.

In [84]:
uog_id = ["UOG1_F","UOG2_F","UOG3_F","UOG4_F","UOG5_F","UOG6_F"]

df_C = df_C_raw[uog_id]
df_C = pd.melt(df_C, value_vars=uog_id)

df_P = df_P_raw[uog_id]
df_P = pd.melt(df_P, value_vars=uog_id)

org_id = pd.concat([df_P, df_C])["value"].unique().tolist()

### Filter ORG Position

In [85]:
df_OP_filter = df_OP[df_OP["ID_POSIZIONE_ORGANIZZATIVA"].isin(org_id)]

,ID_POSIZIONE_ORGANIZZATIVA,UOG,PROVINCIA,REGIONE,PERSONAS_DS,PERSONAS_ID
0,1,IP10510400,Imperia,Liguria,Sales & Relationship Management - Private,P00029
1,2,IP10510700,Novara,Piemonte,Sales & Relationship Management - Private,P00029
2,3,IP10512000,Biella,Piemonte,Sales & Relationship Management - Private,P00029
3,4,IP10520100,Milano,Lombardia,Sales & Relationship Management - Private,P00029
4,5,IP10520200,Milano,Lombardia,Sales & Relationship Management - Private,P00029


### Filter Employee ID by Org ID

### Org Positions

In [88]:
# filter employee ID by org ID
df_name = pd.merge(df_Employees, df_Organization, on = "ID_ORGANIZATIONAL_POSITION")
df_name = df_name.loc[df_name["ID_ORGANIZATIONAL_POSITION"].isin(org_id)]
len(df_name)

9929

In [89]:
df_assess = pd.merge(df_Assessment, df_Domain, on = "ID_KNOWLEDGE_DOMAIN")

In [90]:
# filter employee assessment by org ID
df = pd.merge(df_name, df_assess, on = "ID_EMPLOYEE")
df = df.loc[df["ID_ORGANIZATIONAL_POSITION"].isin(org_id)]
len(df)

MergeError: Must pass "right_on" OR "right_index".

### Create Network File
    1. create nodes

In [ ]:
df_nodes = pd.DataFrame(df["ID_EMPLOYEE"].unique())
df_nodes = df_nodes.rename(columns={0:"Node"})
df_nodes_domain = pd.DataFrame(df["DESCR_KNOWLEDGE_AREA"].unique())
df_nodes_domain = df_nodes_domain.rename(columns={0:"Node"})
df_nodes_domain = df_nodes_domain.dropna()
df_nodes = df_nodes.append(df_nodes_domain)

In [ ]:
nodes = [n for n in df_nodes["Node"].to_numpy()][0:]

In [ ]:
len(nodes)

9963

    2. create edges

In [ ]:
df_edges = df.groupby(["ID_EMPLOYEE","DESCR_KNOWLEDGE_AREA"]).mean()

AGE  \
ID_EMPLOYEE DESCR_KNOWLEDGE_AREA                                  
6           Amministrazione                                52.0   
            Analisi e Gestione del credito                 52.0   
            Business Development                           52.0   
            Capital Markets                                52.0   
            Conoscenza di Prodotti e Servizi Assicurativi  52.0   
...                                                         ...   
68366       Funding                                        49.0   
            Risorse Umane                                  49.0   
            Safety                                         49.0   
            Transaction Banking                            49.0   
            Wealth & Asset Management                      49.0   

                                                           ID_ORGANIZATIONAL_POSITION  \
ID_EMPLOYEE DESCR_KNOWLEDGE_AREA                                                        
6           Amministrazione                                                      55.0   
            Analisi e Gestione del credito                                       55.0   
            Business Development                                                 55.0   
            Capital Markets                                                      55.0   
            Conoscenza di Prodotti e Servizi Assicurativi                        55.0   
...                                                                               ...   
68366       Funding                                                            8763.0   
            Risorse Umane                                                      8763.0   
            Safety                                                             8763.0   
            Transaction Banking                                                8763.0   
            Wealth & Asset Management                                          8763.0   

                                                           ID_MANAGER  \
ID_EMPLOYEE DESCR_KNOWLEDGE_AREA                                        
6           Amministrazione                                   13080.0   
            Analisi e Gestione del credito                    13080.0   
            Business Development                              13080.0   
            Capital Markets                                   13080.0   
            Conoscenza di Prodotti e Servizi Assicurativi     13080.0   
...                                                               ...   
68366       Funding                                             760.0   
            Risorse Umane                                       760.0   
            Safety                                              760.0   
            Transaction Banking                                 760.0   
            Wealth & Asset Management                           760.0   

                                                           SCORE_AUTODETECTION  \
ID_EMPLOYEE DESCR_KNOWLEDGE_AREA                                                 
6           Amministrazione                                                1.0   
            Analisi e Gestione del credito                                 1.0   
            Business Development                                           1.0   
            Capital Markets                                                4.0   
            Conoscenza di Prodotti e Servizi Assicurativi                  1.0   
...                                                                        ...   
68366       Funding                                                        1.0   
            Risorse Umane                                                  1.0   
            Safety                                                         1.0   
            Transaction Banking                                            1.0   
            Wealth & Asset Management                                      1.0   

             

In [ ]:
df_edges = df_edges.reset_index(["ID_EMPLOYEE","DESCR_KNOWLEDGE_AREA"])
df_edges["SCORE_AVG"] = df_edges[["SCORE_AUTODETECTION","SCORE_VALIDATION"]].mean(axis=1)
df_edges = df_edges[["ID_EMPLOYEE","DESCR_KNOWLEDGE_AREA","SCORE_AVG"]]

In [ ]:
weighted_edges = [tuple(n) for n in df_edges.to_numpy()]

In [ ]:
len(df_edges)

129464

    3. create graph

In [ ]:
import networkx as nx

H = nx.Graph()
H.add_nodes_from(nodes)
H.add_weighted_edges_from(weighted_edges)

In [ ]:
print(nx.info(H))

Name: 
Type: Graph
Number of nodes: 9963
Number of edges: 129464
Average degree:  25.9890


    4. add attributes to nodes

In [ ]:
nodes_attr = df.loc[0:,"ID_EMPLOYEE":"ID_PERSONAS"].drop_duplicates()

# add age group
bins = [18, 29, 39, 49, 59, 69, 120]
labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70+']
nodes_attr['AGE GROUPS'] = pd.cut(nodes_attr.AGE, bins, labels = labels, include_lowest = False)

# group attributes with node ID
age_dict = dict(zip(nodes_attr["ID_EMPLOYEE"], nodes_attr["AGE GROUPS"]))
sex_dict = dict(zip(nodes_attr["ID_EMPLOYEE"], nodes_attr["SEX"]))
position_dict = dict(zip(nodes_attr["ID_EMPLOYEE"], nodes_attr["ID_ORGANIZATIONAL_POSITION"]))
provine_dict = dict(zip(nodes_attr["ID_EMPLOYEE"], nodes_attr["PROVINE"]))
region_dict = dict(zip(nodes_attr["ID_EMPLOYEE"], nodes_attr["REGION"]))
persona_dict = dict(zip(nodes_attr["ID_EMPLOYEE"], nodes_attr["ID_PERSONAS"]))

In [ ]:
# add attributes to nodes
# nx.set_node_attributes(H, level_dict, "level")
nx.set_node_attributes(H, age_dict, 'age_groups')
nx.set_node_attributes(H, sex_dict, 'sex')
nx.set_node_attributes(H, provine_dict, 'provine')
nx.set_node_attributes(H, region_dict, 'region')
nx.set_node_attributes(H, persona_dict, 'persona')
nx.set_node_attributes(H, position_dict, 'position')

In [ ]:
# add degrees
degree_dict = dict(H.degree(H.nodes()))
nx.set_node_attributes(H, degree_dict, 'degree')

    5. add attributes to edges

In [ ]:
edges_attr = df.groupby(["ID_EMPLOYEE","DESCR_KNOWLEDGE_AREA"]).mean()
edges_attr = edges_attr.reset_index(["ID_EMPLOYEE","DESCR_KNOWLEDGE_AREA"])
edges_attr = edges_attr[["ID_EMPLOYEE","DESCR_KNOWLEDGE_AREA","SCORE_AUTODETECTION","SCORE_VALIDATION"]]
edges_attr = edges_attr.fillna(0)

In [ ]:
self_score = {}
valid_score = {}

for n in edges_attr[["ID_EMPLOYEE","DESCR_KNOWLEDGE_AREA","SCORE_AUTODETECTION"]].to_numpy():
    a = tuple(n[0:2])
    self_score[a] = {"self_score":n[2]}

for n in edges_attr[["ID_EMPLOYEE","DESCR_KNOWLEDGE_AREA","SCORE_VALIDATION"]].to_numpy():
    a = tuple(n[0:2])
    valid_score[a] = {"valid_score":n[2]}

In [ ]:
# add attributes to edges
nx.set_edge_attributes(H, self_score)
nx.set_edge_attributes(H, valid_score)

In [ ]:
nx.write_graphml(H, "proto_data.graphml")

## Create Tree Data

In [ ]:
df_tree = df_C.append(df_P)
df_tree = df_tree.replace({"-":None})
df_tree.to_csv("tree.csv")

In [ ]:
df_C_raw.head()

,UOG Eff. (Codice HR),Società Eff.,UOG Eff. LE 01,UOG1_F,UOG Eff. LE 01 (DesM),UOG Eff. LE 02,UOG2_F,UOG Eff. LE 02 (DesM),UOG Eff. LE 03,UOG3_F,UOG Eff. LE 03 (DesM),UOG Eff. LE 04,UOG4_F,UOG Eff. LE 04 (DesM),UOG Eff. LE 05,UOG5_F,UOG Eff. LE 05 (DesM),UOG Eff. LE 06,UOG6_F,UOG Eff. LE 06 (DesM)
0,BG71220Z13,2000-INTESA SANPAOLO S.P.A.,BG17827000,5827,AREA GOV CIDIO,BG71220000,8253,CIDIO DIR CEN OPERAT,BG71220002,8237,DC OP OPERAT FINANZA,BG71220013,8275,DC OP DOCUMENTATION,BG71220Z13,8338,DC OP DOCUMENTATION,senza attrib.,-,2019/5/Senza attribuz.
1,BG71220013,2000-INTESA SANPAOLO S.P.A.,BG17827000,5827,AREA GOV CIDIO,BG71220000,8253,CIDIO DIR CEN OPERAT,BG71220002,8237,DC OP OPERAT FINANZA,BG71220013,8275,DC OP DOCUMENTATION,senza attrib.,-,2019/5/Senza attribuz.,senza attrib.,-,2019/5/Senza attribuz.
2,BG71220U41,2000-INTESA SANPAOLO S.P.A.,BG17827000,5827,AREA GOV CIDIO,BG71220000,8253,CIDIO DIR CEN OPERAT,BG71220008,8256,DC OP TRANSAZIONALE,BG71220041,8290,DC OP TESORERIA ENTI,BG71220U41,8414,DC OP TESORERIA ENTI,senza attrib.,-,2019/5/Senza attribuz.
3,BG19562V11,2000-INTESA SANPAOLO S.P.A.,BIG6917000,860,AREA GOV COO,BG19562000,6230,DC DIR POL SVI LE AC,BG19562006,8091,DC SL PHYG LEAR DES,BG19562011,8093,DC SL LIVE LEARNING,BG19562V11,9736,DC SL LIVE LEARNING,senza attrib.,-,2019/5/Senza attribuz.
4,BG71000022,2000-INTESA SANPAOLO S.P.A.,BIG6917000,860,AREA GOV COO,BG71000000,8034,COO DIR CEN ORGANIZ,BG71000006,8053,DC OR PEOPL PROC CAR,BG71000022,8058,DC OR SERVIZI GENER,senza attrib.,-,2019/5/Senza attribuz.,senza attrib.,-,2019/5/Senza attribuz.


In [ ]:
# add tree values
df_tree_value = dict(zip(df_C_raw["UOG1_F"],df_C_raw["UOG Eff. LE 01 (DesM)"]))
df_tree_value_2 = dict(zip(df_C_raw["UOG2_F"],df_C_raw["UOG Eff. LE 02 (DesM)"]))
df_tree_value_3 = dict(zip(df_C_raw["UOG3_F"],df_C_raw["UOG Eff. LE 03 (DesM)"]))
df_tree_value_4 = dict(zip(df_C_raw["UOG4_F"],df_C_raw["UOG Eff. LE 04 (DesM)"]))
df_tree_value_5 = dict(zip(df_C_raw["UOG5_F"],df_C_raw["UOG Eff. LE 05 (DesM)"]))
df_tree_value_6 = dict(zip(df_C_raw["UOG6_F"],df_C_raw["UOG Eff. LE 06 (DesM)"]))

df_tree_value.update(df_tree_value_2)
df_tree_value.update(df_tree_value_3)
df_tree_value.update(df_tree_value_4)
df_tree_value.update(df_tree_value_5)
df_tree_value.update(df_tree_value_6)

df_tree_value_7 = dict(zip(df_P_raw["UOG1_F"],df_P_raw["UOG Eff. LE 01 (DesM)"]))
df_tree_value_8 = dict(zip(df_P_raw["UOG2_F"],df_P_raw["UOG Eff. LE 02 (DesM)"]))
df_tree_value_9 = dict(zip(df_P_raw["UOG3_F"],df_P_raw["UOG Eff. LE 03 (DesM)"]))
df_tree_value_10 = dict(zip(df_P_raw["UOG4_F"],df_P_raw["UOG Eff. LE 04 (DesM)"]))
df_tree_value_11 = dict(zip(df_P_raw["UOG5_F"],df_P_raw["UOG Eff. LE 05 (DesM)"]))
df_tree_value_12 = dict(zip(df_P_raw["UOG6_F"],df_P_raw["UOG Eff. LE 06 (DesM)"]))

df_tree_value.update(df_tree_value_7)
df_tree_value.update(df_tree_value_8)
df_tree_value.update(df_tree_value_9)
df_tree_value.update(df_tree_value_10)
df_tree_value.update(df_tree_value_11)
df_tree_value.update(df_tree_value_12)

In [ ]:
import csv
from collections import defaultdict

In [ ]:
def ctree():
    """ One of the python gems. Making possible to have dynamic tree structure.

    """
    return defaultdict(ctree)

def build_leaf(name, leaf):
    """ Recursive function to build desired custom tree structure
    """
    res = {"name": name}
    res["position_name"] = df_tree_value.get(float(name))
        
    # add children node if the leaf actually has any children
    if len(leaf.keys()) > 0:
        a = []
        for k, v in leaf.items():
            if k != "":
                a.append(build_leaf(k,v)) 
        res["children"]= a
    return res

In [ ]:
""" The main thread composed from two parts.

First it's parsing the csv file and builds a tree hierarchy from it.
Second it's recursively iterating over the tree and building custom
json-like structure (via dict).

And the last part is just printing the result.

"""
tree = ctree()
# NOTE: you need to have test.csv file as neighbor to this file
with open('tree.csv') as csvfile:
    reader = csv.reader(csvfile)
    for rid, row in enumerate(reader):

        # skipping first header row. remove this logic if your csv is
        # headerless
        if rid == 0:
            continue

        # usage of python magic to construct dynamic tree structure and
        # basically grouping csv values under their parents
        leaf = tree[row[0]]
        for cid in range(1, len(row)):
            leaf = leaf[row[cid]]

# building a custom tree structure
res = []
for name, leaf in tree.items():
#     print(name)
    a = build_leaf(name, leaf)
    res.append(a)

In [ ]:
res

[{'name': '5827',
  'position_name': 'AREA GOV CIDIO',
  'children': [{'name': '8253',
    'position_name': 'CIDIO DIR CEN OPERAT',
    'children': [{'name': '8237',
      'position_name': 'DC OP OPERAT FINANZA',
      'children': [{'name': '8275',
        'position_name': 'DC OP DOCUMENTATION',
        'children': [{'name': '8338',
          'position_name': 'DC OP DOCUMENTATION',
          'children': []},
         {'name': '8339',
          'position_name': 'DC OP DOCUMENTATION',
          'children': []}]},
       {'name': '8239',
        'position_name': 'DC OP CAMBI DERIVATI',
        'children': [{'name': '8335',
          'position_name': 'DC OP CAMBI DERIVATI',
          'children': []},
         {'name': '8334',
          'position_name': 'DC OP CAMBI DERIVATI',
          'children': []},
         {'name': '8336',
          'position_name': 'DC OP CAMBI DERIVATI',
          'children': []}]},
       {'name': '8222', 'position_name': 'DC OP TITOLI', 'children': []},
       {'n

In [ ]:
import json
with open('tree.json', 'w') as outfile:
    json.dump(res, outfile, sort_keys=True, indent=4)